In [12]:
import yahoo_fin.stock_info as yf
import pandas as pd
from stage import fullPrint
import random
from datetime import datetime, timedelta
import numpy as np
nyseDf = pd.read_html('stockList.html')
nyseDf = pd.DataFrame(nyseDf[0][0])
nyseList = []
for row in nyseDf.iterrows():
    ticker = str(row[1]).split()[1]
    if ticker == "Symbol":
        continue
    nyseList.append(ticker)
allList = []
nasdaqList = yf.tickers_nasdaq()
allList = nasdaqList + nyseList
tenYearSum = [0 for i in range(7)]
curSum = [0 for i in range(7)]
j = 0
random.shuffle(allList)
listOfStocks = [[] for i in range(4)]


In [13]:
for i in range(len(allList)):
    listOfStocks[i%4].append(allList[i])
    # try:
    #     df = pd.read_pickle("stockData/nyseNasdaq/"+allList[i]+".pkl")
    # except:
    #     pass
    # try:
    #     # drift = df.index[0].weekday()
    #     # start = pd.to_datetime('2018-02-12',format = '%Y-%m-%d')
    #     # end = pd.to_datetime('2022-01-10',format = '%Y-%m-%d')
    #     # start = start + timedelta(days= drift)
    #     # end = end + timedelta(days=drift)
    #     a = df.iat[0,0]
    #     b = df.iat[-1,0]
    #     length = len(df.index)/52
    #     # print(j%7,": first:",df.at[pd.to_datetime('2012-01-09',format = '%Y-%m-%d'),'close'],"second:",df.at[pd.to_datetime('2022-01-10',format = '%Y-%m-%d'),'close'])
    #     if a != 0 and length > 3:
    #         curSum[j%4] += (b/a)**(1/length)
    #         print((b/a)**(1/length))
    #         j = j + 1
    #     # print(j%7,curSum[j%7], tenYearSum[j%7])
        
    # except:
    #     print(allList[i],"did not exist then")

In [14]:
def calculateGroupReturn(list):
    monthlyReturn = pd.read_pickle("transactionTemplate.pkl")
    monthlyReturn = monthlyReturn.resample('M').mean()
    monthlyReturn.index = monthlyReturn.index.strftime("%Y:%m")
    currentColumn = -1
    for symbol in list:
        if symbol in monthlyReturn.columns:
            continue
        try:
            df = pd.read_pickle("stockData/nyseNasdaq/"+symbol+".pkl")
        except:
            continue
        if not (df['close'] != 0).all():
            continue
        monthlyReturn[symbol] = 1
        currentColumn += 1
        monthCheck = df.index[0].strftime("%Y:%m")
        maxPrice = df.iat[0,0]
        initialPrice = df.iat[0,0]
        for index, element in df.iterrows():
            
            if (index.strftime("%Y:%m") != monthCheck):
                if (maxPrice / initialPrice >= 1.03):
                    rowNumber = monthlyReturn.index.get_loc(index.strftime("%Y:%m"))
                    monthlyReturn.iat[rowNumber, currentColumn] = (maxPrice / initialPrice)
                initialPrice = element[0]
                maxPrice = element[0]
                monthCheck = index.strftime("%Y:%m")
            if (element[0] > maxPrice): maxPrice = element[0]
    return monthlyReturn
            
            
            

In [15]:
import warnings
warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning) 
i = 0
listOfDf = []
for l in listOfStocks:
    listOfDf.append(calculateGroupReturn(l))
print(listOfDf)


[              LNT  NRC  PHIC       GM       PHX      GNRC      CSII      VEON  \
1995:01  1.000000  1.0     1  1.00000  1.000000  1.000000  1.000000  1.000000   
1995:02  1.081081  1.0     1  1.00000  1.000000  1.000000  1.000000  1.000000   
1995:03  1.000000  1.0     1  1.00000  1.000000  1.000000  1.000000  1.000000   
1995:04  1.000000  1.0     1  1.00000  1.000000  1.000000  1.000000  1.000000   
1995:05  1.000000  1.0     1  1.00000  1.062499  1.000000  1.000000  1.000000   
...           ...  ...   ...      ...       ...       ...       ...       ...   
2021:09  1.000000  1.0     1  1.00000  1.000000  1.093268  1.000000  1.204545   
2021:10  1.000000  1.0     1  1.07355  1.164234  1.000000  1.038014  1.000000   
2021:11  1.000000  1.0     1  1.00000  1.000000  1.225626  1.082716  1.000000   
2021:12  1.000000  1.0     1  1.08339  1.000000  1.000000  1.000000  1.000000   
2022:01  1.043811  1.0     1  1.00000  1.000000  1.000000  1.057508  1.000000   

              MLP        V

In [32]:
print(listOfDf[0]['LNT'].product())
product = 1
growthOfStock = {}
index = 0
for l in listOfStocks:
    for ticker in l:
        if ticker in listOfDf[index].columns:
            growthOfStock[ticker] = listOfDf[index].product()
    index += 1

774.4023284837486
1.0 1.0
1.0810810810810811 1.0810810810810811
1.0810810810810811 1.0
1.0810810810810811 1.0
1.0810810810810811 1.0
1.139517896274653 1.054054054054054
1.1795009803544656 1.0350877192982457
1.1795009803544656 1.0
1.1795009803544656 1.0
1.1795009803544656 1.0
1.2191480721310863 1.0336134453781514
1.2191480721310863 1.0
1.2191480721310863 1.0
1.2191480721310863 1.0
1.2191480721310863 1.0
1.2191480721310863 1.0
1.2191480721310863 1.0
1.2191480721310863 1.0
1.319489477244756 1.0823045267489713
1.319489477244756 1.0
1.319489477244756 1.0
1.319489477244756 1.0
1.319489477244756 1.0
1.319489477244756 1.0
1.319489477244756 1.0
1.319489477244756 1.0
1.319489477244756 1.0
1.319489477244756 1.0
1.319489477244756 1.0
1.319489477244756 1.0
1.319489477244756 1.0
1.319489477244756 1.0
1.319489477244756 1.0
1.3813405464906041 1.046875
1.3813405464906041 1.0
1.471821368313133 1.0655021834061136
1.5919700514407358 1.0816326530612246
1.5919700514407358 1.0
1.5919700514407358 1.0
1.659844

In [24]:
from stage import fullPrint
fullPrint(listOfDf[0]['LNT'])

1995:01    1.000000
1995:02    1.081081
1995:03    1.000000
1995:04    1.000000
1995:05    1.000000
1995:06    1.054054
1995:07    1.035088
1995:08    1.000000
1995:09    1.000000
1995:10    1.000000
1995:11    1.033613
1995:12    1.000000
1996:01    1.000000
1996:02    1.000000
1996:03    1.000000
1996:04    1.000000
1996:05    1.000000
1996:06    1.000000
1996:07    1.082305
1996:08    1.000000
1996:09    1.000000
1996:10    1.000000
1996:11    1.000000
1996:12    1.000000
1997:01    1.000000
1997:02    1.000000
1997:03    1.000000
1997:04    1.000000
1997:05    1.000000
1997:06    1.000000
1997:07    1.000000
1997:08    1.000000
1997:09    1.000000
1997:10    1.046875
1997:11    1.000000
1997:12    1.065502
1998:01    1.081633
1998:02    1.000000
1998:03    1.000000
1998:04    1.042636
1998:05    1.041045
1998:06    1.000000
1998:07    1.000000
1998:08    1.000000
1998:09    1.053996
1998:10    1.111814
1998:11    1.000000
1998:12    1.000000
1999:01    1.000000
1999:02    1.000000
